# LeagueRank: A game skill evaluatioin application
## Introduction

[add words here...]
[some terminology...]

## Content
[Topics gonna cover in this project...]

## Installing the library

Before getting started, we need to prepare some usful libraries. The main package we use to crawl the LOL data is called ```riotwatcher```, which is a [Python library](https://github.com/pseudonym117/Riot-Watcher) built up with the API provided by Riot. To install the library, one common way using ```pip```:

```pip install riotwatcher```

Although the library is robust and convenience, there are still some functions it does not implement such as getting the most frequently played champion, getting summoner list by division, etc. To implement those functions, we are going to adopt corresponding Riot APIs, using urllib2 to request from the game server. Installing urllib2 via ```pip```:

```pip install urllib2```

After you run all the installs, make sure the following commands work for you

In [1]:
import csv
import json
import os
import ujson
import urllib2

from riotwatcher import RiotWatcher

To use the Riot api, one more important thing to do is to get your own API key. API key can be obtained from [here](https://developer.riotgames.com/). Note that normal developr API key has a narrow request limit, whereas production API key for commercial use has a looser requirement of request limit. For now, we are just gonna use the normal API key for demonstration.

After getting your own API key, put it in the config dictionary below:

In [2]:
config = {
    'key': ['enter your api key here'],
}

## Project Architecture
### Data Crawling

#### 1. Fetch player list

[how to crawl the data...]

In [4]:
class RiotCrawler:
    def __init__(self, key):
        self.key = key
        self.w = RiotWatcher(key)
        self.division = {
            'bronze': [],
            'silver': [],
            'gold': [],
            'platinum': [],
            'diamond': [],
            'challenger': [],
            'master': [],
        }

    # def get_player_list(self):
    #     recent_games = self.w.get_recent_games(self.player_id)
    #     player_list = set()
    #     for game in recent_games['games']:
    #         # only pick up ranked games
    #         if 'RANKED' in game['subType']:
    #             fellow_players = game['fellowPlayers']
    #             for fellow_player in fellow_players:
    #                 fellow_player_id = fellow_player['summonerId']
    #                 if fellow_player_id not in player_list:
    #                     player_list.add(fellow_player_id)
    #     return list(player_list)

    def get_player_by_division(self, summoner_id):
        request_url = 'https://na.api.pvp.net/api/lol/na/v2.5/league/by-summoner/{}?api_key={}'.format(
            summoner_id, self.key
        )
        response = urllib2.urlopen(request_url)
        division_info = ujson.loads(response.read())

        tier = division_info[str(summoner_id)][0]['tier'].lower()
        entries = division_info[str(summoner_id)][0]['entries']

        level = self.division[tier]
        # cnt = 0
        for entry in entries:
            level.append(entry['playerOrTeamId'])
            # cnt += 1
            # if cnt == 3:
            #     break

        # for l in level:
        #     print 'summoner id: {}, name: {}'.format(str(l.id), l.name)


def get_division():
    # challenger: 77759242
    # platinum: 53381
    # gold: 70359816
    # silver: 65213225
    # bronze: 22309680
    # master: 22551130
    # diamond: 34570626
    player_ids = [70359816, 77759242, 53381, 65213225, 22309680, 22551130, 34570626]
    riot_crawler = RiotCrawler(config['key'])
    for player_id in player_ids:
        print 'start crawling id: {}'.format(player_id)
        riot_crawler.get_player_by_division(player_id)
    return riot_crawler.division


# for tier, rank_dict in riot_crawler.division.iteritems():
#     print '--- {} ---'.format(tier)
#     for summoner in rank_dict:
#         print 'summoner id: {}'.format(summoner)
#     print '--- end of {} ---'.format(tier)

#### 2. Fetch most frequently played champions

[Insert words here...]